# Llama 2를 사용한 검색 증강 생성

이 노트북은 Llama 2를 사용하여 (컨텍스트 내) 검색 증강 생성을 수행하는 방법을 안내합니다. 모델이 제공된 컨텍스트만 사용하여 응답을 생성하도록 Llama 2의 `system` 메시지를 사용자 지정합니다.

**컨텍스트 내 검색 증강 생성이란 무엇입니까?**


컨텍스트 내 검색 증강 생성은 관련 문서를 모델 입력에 포함하여 언어 모델 생성을 개선하는 방법입니다. 핵심 사항은 다음과 같습니다.

* 모델에 대한 사실적 근거를 제공하기 위해 외부 코퍼스에서 관련 문서를 검색합니다.
* 모델 아키텍처를 수정하거나 모델을 미세 조정하지 않고 검색된 문서를 입력 텍스트 앞에 추가합니다.
* 기성품 고정 언어 모델을 사용하여 외부 지식을 활용할 수 있습니다.

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

## 간단한 예제

다음은 기존 llama2 프롬프트 빌더를 사용하여 프롬프트를 생성하는 간단한 예제입니다. [llama-index](https://gpt-index.readthedocs.io/en/latest/examples/vector_stores/SimpleIndexDemoLlama-Local.html)의 `system` 메시지를 약간 수정하여 사용할 것입니다.

In [8]:
SYSTEM_PROMPT = """당신은 주어진 #SOURCE# 문서를 기반으로 친근한 방식으로 질문에 답변하는 AI 조수입니다. 항상 따르는 몇 가지 규칙은 다음과 같습니다.
- 사람이 읽을 수 있는 출력을 생성하고, 의미 없는 텍스트로 출력을 생성하지 마십시오.
- 요청된 출력만 생성하고, 요청된 출력 앞이나 뒤에 다른 언어를 포함하지 마십시오.
- 절대 감사하다고 말하거나, 기꺼이 도와드리겠다고 말하거나, 자신이 AI 에이전트라고 말하는 등의 말을 하지 마십시오. 직접 답변하십시오.
- 북미 비즈니스 문서에서 일반적으로 사용되는 전문적인 언어를 생성하십시오.
- 절대 공격적이거나 불쾌한 언어를 생성하지 마십시오.
- #SOURCE# 문서에 기반한 사실과 정보만 포함하십시오.
"""

system = {"role": "system", "content": SYSTEM_PROMPT}

이제 LLM을 호출하기 전에 `query`와 `context`가 포함된 사용자 지침을 만들어 보겠습니다. 컨텍스트로는 [뉘른베르크의 위키백과 문서](https://en.wikipedia.org/wiki/Nuremberg)(제가 사는 도시)를 복사했습니다.
_노트북을 오염시키지 않기 위해 요지로 업로드했습니다._

In [ ]:
!wget https://gist.githubusercontent.com/philschmid/2678351cb9f41d385aa5c099caf20c0a/raw/60ae425677dd9bed6fe3c0f2dd5b6ea49bc6590c/nuremberg.txt

In [14]:
context = open("nuremberg.txt").read()

query = "뉘른베르크에는 몇 명이 살고 있습니까?"

컨텍스트를 사용하기 전에 모델에게 물어봅시다.

In [15]:
from easyllm.clients import huggingface

# 프롬프트 빌더를 llama2로 설정합니다.
huggingface.prompt_builder = "llama2"
# huggingface.api_key = "hf_xx"

# ChatCompletion 요청을 보냅니다.
response = huggingface.ChatCompletion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    messages=[
        {"role": "user", "content": query},
    ],
)

# 시간 지연 및 수신된 텍스트를 인쇄합니다.
print(response["choices"][0]["message"]["content"])


 2020년 12월 31일 현재 독일 뉘른베르크의 인구는 약 516,000명입니다.


이제 `context`와 함께 `system` 메시지를 사용하여 모델의 지식을 "메모리 내"에서 보강하고 동일한 질문을 다시 해보겠습니다.

In [23]:
context_extended = f"{query}\n\n#SOURCE#\n{context}"
# context_extended = f"{query}\n\n#SOURCE START#\n{context}\n#SOURCE END#{query}"

In [22]:
from easyllm.clients import huggingface

# 프롬프트 빌더를 llama2로 설정합니다.
huggingface.prompt_builder = "llama2"
# huggingface.api_key = "hf_xx"

# ChatCompletion 요청을 보냅니다.
response = huggingface.ChatCompletion.create(
    model="meta-llama/Llama-2-70b-chat-hf",
    messages=[
        system, 
        {"role": "user", "content": context_extended},
    ],
)

# 시간 지연 및 수신된 텍스트를 인쇄합니다.
print(response["choices"][0]["message"]["content"])


 2022-12-31 데이터에 따르면 뉘른베르크의 인구는 523,026명입니다.


훌륭합니다! 요지를 확인하면 다음과 같은 스니펫이 있습니다.
```bash
인구 (2022-12-31)[2]
 • 도시	523,026
```

## 다음 단계

다음 단계는 도구 및 API 또는 벡터 데이터베이스 및 임베딩을 사용하여 위키, 웹 또는 기타 데이터베이스와 같은 외부 지식 소스와 LLM을 연결하는 것입니다.